In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
from collections import defaultdict
import time
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

lol_watcher = LolWatcher('RGAPI-a73e9ee3-61bb-4e38-8f48-998ff2478d88')
my_region = 'na1'

In [2]:
# Champion base stat
latest = lol_watcher.data_dragon.versions_for_region(my_region)['n']['champion']
champ = lol_watcher.data_dragon.champions(latest, False, 'en_US')

champs = pd.DataFrame.from_dict(champ['data'], orient='index')
champs = pd.DataFrame(champs.stats.values.tolist(), index=champs['id'])

champ_stats = {}
for champion in champ['data']:
    champ_stats[champion] = champ['data'][champion]['stats']

champ_stats_df = pd.DataFrame.from_dict(champ_stats, orient = 'index')
champ_stats_df.rename(index={'Fiddlesticks':'FiddleSticks'}, inplace=True)
champ_stats_df.head()

,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
Aatrox,580.0,90,0.0,0.0,345,38,3.25,32.0,1.25,175,3.00,1.00,0.000,0.0,0,0,60.0,5.00,2.500,0.651
Ahri,500.0,82,418.0,25.0,330,18,3.50,30.0,0.50,550,2.50,0.60,8.000,0.8,0,0,53.0,3.00,2.000,0.668
Akali,500.0,105,200.0,0.0,345,23,3.50,37.0,1.25,125,9.00,0.90,50.000,0.0,0,0,62.0,3.30,3.200,0.625
Akshan,560.0,90,350.0,40.0,330,26,3.00,30.0,0.50,500,3.75,0.65,8.175,0.7,0,0,52.0,3.50,4.000,0.638
Alistar,600.0,106,350.0,40.0,330,44,3.50,32.0,1.25,125,8.50,0.85,8.500,0.8,0,0,62.0,3.75,2.125,0.625


In [3]:
summoner_names_input = '구l두라미,sanspareil2,tongxinjishou,월하무즉정야,T1 CrazzyCar'
summoner_names = summoner_names_input.split(sep=',')
summoner_ids = {lol_watcher.summoner.by_name('kr', summoner)['puuid']:summoner for summoner in summoner_names}

In [4]:
player_stats = []
region = 'asia'
for id, name in tqdm_notebook(summoner_ids.items()):
    matchlist = lol_watcher.match.matchlist_by_puuid(region, id, start = 0, count = 20)
    for match in tqdm_notebook(matchlist, leave = False):
        match_info = lol_watcher.match.by_id(region, match)
        for player in match_info['info']['participants']:
            if player['puuid'] == id:
                player['summonerName'] = name
                player_stats.append(player)

player_stats_df = pd.DataFrame(player_stats)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
player_stats_df_summary = player_stats_df.groupby(['summonerName', 'championName']).mean()
player_stats_df_summary.drop(columns=['championId'], inplace=True)
player_stats_df_count = player_stats_df.groupby(['summonerName', 'championName']).size()
player_stats_df_summary = player_stats_df_summary.join(pd.DataFrame(player_stats_df_count, columns = ['count']))
player_stats_df_summary.reset_index(level = 0, inplace=True)

In [6]:
# Combine player stat and champion base stat
player_combined = player_stats_df_summary.join(champ_stats_df)
player_combined = player_combined.reset_index().rename(columns={'index':'championName'})
player_combined.summonerName.unique()

array(['sanspareil2', 'tongxinjishou', '구l두라미', 'T1 CrazzyCar', '월하무즉정야'],
      dtype=object)

In [7]:
summoner_names

['구l두라미', 'sanspareil2', 'tongxinjishou', '월하무즉정야', 'T1 CrazzyCar']

In [9]:
champ_selection_input = 'Ryze,Volibear,Alistar,Yasuo,Draven'
champ_selection = champ_selection_input.split(sep=',')
current_team = pd.DataFrame(
    {'summonerName': summoner_names, 'championName':champ_selection}
)
current_team_stats = pd.merge(current_team, player_combined, 'left', on = ['summonerName', 'championName']).fillna(0)

In [10]:
current_team_stats

,summonerName,championName,assists,baronKills,bountyLevel,champExperience,champLevel,championTransform,consumablesPurchased,damageDealtToBuildings,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,구l두라미,Ryze,8.5,0.000000,1.000000,12687.500000,14.000000,0.0,1.500000,2154.000000,...,8.00,0.8,8.00,1.00,0.0,0.0,58.0,3.00,2.112,0.625
1,sanspareil2,Volibear,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.000,0.000
2,tongxinjishou,Alistar,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.000,0.000
3,월하무즉정야,Yasuo,5.0,0.000000,1.153846,10495.461538,12.461538,0.0,3.538462,2494.461538,...,6.50,0.9,0.00,0.00,0.0,0.0,60.0,3.00,3.500,0.697
4,T1 CrazzyCar,Draven,5.0,0.142857,1.714286,12258.714286,14.000000,0.0,3.142857,7463.714286,...,3.75,0.7,8.04,0.65,0.0,0.0,60.0,3.61,2.700,0.679


In [11]:
current_team_out = current_team_stats.drop(columns=['summonerName', 'championName', 'win', 'count']).stack()
current_team_out.index = current_team_out.index.map('{0[1]}_{0[0]}'.format)
current_team_out = current_team_out.to_frame().T
current_team_out.reindex(sorted(current_team_out.columns), axis=1)

,armor_0,armor_1,armor_2,armor_3,armor_4,armorperlevel_0,armorperlevel_1,armorperlevel_2,armorperlevel_3,armorperlevel_4,...,wardsKilled_0,wardsKilled_1,wardsKilled_2,wardsKilled_3,wardsKilled_4,wardsPlaced_0,wardsPlaced_1,wardsPlaced_2,wardsPlaced_3,wardsPlaced_4
0,22.0,0.0,0.0,30.0,29.0,3.0,0.0,0.0,3.4,3.3,...,0.5,0.0,0.0,1.692308,4.142857,6.5,0.0,0.0,5.307692,7.714286


In [12]:
training_dataset = pd.read_csv('training_dataset.csv')

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

x_train = training_dataset.reindex(sorted(training_dataset.columns), axis=1).drop(columns = 'win_')
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)

y_train = training_dataset['win_']
x_test = current_team_out.reindex(sorted(current_team_out.columns), axis=1)
x_test_scaled = scaler.transform(x_test)
logit = LogisticRegression(max_iter=50000)

In [14]:
logit.fit(x_train_scaled, y_train)
logit.predict(x_test_scaled)

array([False])

In [15]:
logit.predict_proba(x_test_scaled)

array([[0.9964748, 0.0035252]])

In [16]:
rf = RandomForestClassifier()
rf.fit(x_train_scaled, y_train)
rf.predict(x_test_scaled)

array([ True])

In [17]:
rf.predict_proba(x_test_scaled)

array([[0.38, 0.62]])